# BeatSaver API usage and Data Processing notebook

This notebook is for developing code associated with downloading Beat Saber custom songs and mappings from the BeatSaver API. Goals include:

- Systematically download songs, maps, and associated metadata
- Process data
- Save data to database